In [2]:
#References are taken from Restaurant Review Project(https://www.geeksforgeeks.org/python-nlp-analysis-of-restaurant-reviews/)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [24]:
df = pd.read_excel('Women_nlp.xlsx')
df.head()#Orignial data set

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1


In [25]:
df.shape

(23488, 2)

In [26]:
df.isna().sum()  #We are getting 845 Review Text with NAN cluln , we need to remove the same

Review Text        845
Recommended IND      0
dtype: int64

In [33]:

#Drop row columns
df1 = df.dropna() 
df1.head()




,level_0,index,Review Text,Recommended IND
0,0,0,Absolutely wonderful - silky and sexy and comf...,1
1,1,1,Love this dress! it's sooo pretty. i happene...,1
2,2,2,I had such high hopes for this dress and reall...,0
3,3,3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,4,4,This shirt is very flattering to all due to th...,1


In [36]:
df1.isna().sum() #no nan value in Reviews

level_0            0
index              0
Review Text        0
Recommended IND    0
dtype: int64

In [37]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22643 entries, 0 to 22642
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   level_0          22643 non-null  int64 
 1   index            22643 non-null  int64 
 2   Review Text      22643 non-null  object
 3   Recommended IND  22643 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 884.5+ KB


In [38]:
df.shape

(22643, 4)

In [67]:
# library to clean data
import re
 
# Natural Language Tool Kit
import nltk
 
nltk.download('stopwords')
 
# to remove stopword
from nltk.corpus import stopwords
 
# for Stemming propose
from nltk.stem.porter import PorterStemmer
 
# Initialize empty array
# to append clean text
corpus = []
 
# 22643 (reviews) rows to clean
for i in range(0, 22643):
     
    # column : "Review", row ith
    review = re.sub('[^a-zA-Z]', ' ', df1['Review Text'][i])
     
    # convert all cases to lower cases
    review = review.lower()
     
    # split to array(default delimiter is " ")
    review = review.split()
     
    # creating PorterStemmer object to
    # take main stem of each word
    ps = PorterStemmer()
     
    # loop for stemming each word
    # in string array at ith row   
    review = [ps.stem(word) for word in review
                if not word in set(stopwords.words('english'))]
                 
    # rejoin all string array elements
    # to create back into a string
    review = ' '.join(review)
     
    # append each string to create
    # array of clean text
    corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [68]:
print(corpus)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [69]:
#Creating Bag of Models

from sklearn.feature_extraction.text import CountVectorizer  
#To extract max 1500 feature,"max_features" is attribute to experiment with to get better results
cv = CountVectorizer(max_features=1500)  #Max feature is nothing but number of imp words are categorized by model on which our model is being trained.Our model categorized 9000 words while we check max feature on 9000 workds we are getting poor accurcay of 40%,while we checked model on 1000 feature we get 73% accuracy.If we further reduce the max feature to 100 words we getting accuracy of 76%
X = cv.fit_transform(corpus).toarray()
# y contains answers if review is positive or negative ,"0" for negative and "1" for Positive
y = df1.iloc[:, -1].values

In [74]:
#finding words in X(review Column)

len(X[0])  #We got 1500 words taken from all the reviews with the help of tokenization, We will take 9000 words for our bag of words model.

1500

In [75]:
#Splitting model in train and test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [76]:
#Train Naives bayes model on Training Set

#from sklearn.naive_bayes import GaussianNB
#classifier = GaussianNB()
#classifier.fit(X_train, y_train)


In [78]:
#Train set result,73 %accuracy on max feature of 1000,feel free to change max feature in naives bayes to check accuracy on another max feature


#y_pred = classifier.predict(X_train)
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_train.reshape(len(y_train),1)),1))

#Confusion matrix for Train Set
#from sklearn.metrics import confusion_matrix, accuracy_score
#cm = confusion_matrix(y_train, y_pred)
#print(cm)
#accuracy_score(y_train, y_pred)

# Fitting Random Forest Classification
# to the Training set
from sklearn.ensemble import RandomForestClassifier
 
# n_estimators can be said as number of
# trees, experiment with n_estimators
# to get better results
model = RandomForestClassifier(n_estimators = 501,
                            criterion = 'entropy')
                             
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=501,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [81]:

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
 
cm = confusion_matrix(y_test, y_pred)
 
cm

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 372  464]
 [1573 2120]]


0.5502318392581144

# Natural Language Processing

## Importing the libraries

In [86]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [85]:
dataset = pd.read_excel('Women_nlp.xlsx')

In [88]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23488 entries, 0 to 23487
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Review Text      22643 non-null  object
 1   Recommended IND  23488 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 367.1+ KB


In [89]:
dataset.isna().sum()

Review Text        845
Recommended IND      0
dtype: int64

In [100]:
#Drop row columns
dataset1 = df.dropna() 
dataset1.head()
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22643 entries, 0 to 22642
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   level_0          22643 non-null  int64 
 1   index            22643 non-null  int64 
 2   Review Text      22643 non-null  object
 3   Recommended IND  22643 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 884.5+ KB


## Cleaning the texts

In [101]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0,22643 ):
  review = re.sub('[^a-zA-Z]', ' ', dataset1['Review Text'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [102]:
print(corpus)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Creating the Bag of Words model

In [109]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 100)# We select only 100 words as max features and Accuracy increased to 75% 
X = cv.fit_transform(corpus).toarray()
y = dataset1.iloc[:, -1].values

## Splitting the dataset into the Training set and Test set

In [110]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

## Training the Naive Bayes model on the Training set

In [111]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

## Predicting the Test set results

In [112]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [1 1]
 [1 1]
 ...
 [0 0]
 [1 1]
 [1 1]]


## Making the Confusion Matrix

In [113]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 649  208]
 [ 884 2788]]


0.758887171561051